<a href="https://colab.research.google.com/github/gilvandrocesardemedeiros/CCD/blob/master/Projeto_Final%5BMotor%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
corrosao_diag = pd.read_excel("https://github.com/gilvandrocesardemedeiros/DIMAP-BigData/raw/master/Data/Motor_original.xls.xlsx")

In [0]:
dataBase = corrosao_diag.dropna().reset_index(drop = True)

In [0]:
print("Dados perdidos [Diagnóstico == NaN]: " + str(len(corrosao_diag) - len(dataBase)))

Dados perdidos [Diagnóstico == NaN]: 60


In [0]:
dataBase.describe()

,Km Atual,Km Troca,Fe,Cr,Pb,Cu,Sn,Al,Ni,Ag,Si,B,Na,Mg,Ca,Ba,P,Zn,Mo,Ti,V,K
count,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.00000,725.000000,725.000000,725.000000,725.000000,725.00000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000
mean,317346.384828,10903.004138,97.377931,3.628966,11.582069,6.880000,4.183448,8.49931,0.748966,0.044138,10.314483,0.228966,168.22069,471.393103,1676.797241,0.200000,894.340690,1134.522759,6.841379,0.001379,0.001379,4.491034
std,121214.743593,4718.952108,48.786892,3.126520,9.176324,6.496127,2.105177,4.05270,0.884420,0.394345,3.985869,2.514596,108.75848,300.390259,704.071919,2.164505,225.798848,292.938359,5.409318,0.037139,0.037139,3.755878
min,27752.000000,1619.000000,11.000000,0.000000,0.000000,1.000000,0.000000,1.00000,0.000000,0.000000,4.000000,0.000000,1.00000,15.000000,452.000000,0.000000,268.000000,340.000000,0.000000,0.000000,0.000000,0.000000
25%,254186.000000,8742.000000,68.000000,2.000000,7.000000,4.000000,3.000000,6.00000,0.000000,0.000000,8.000000,0.000000,32.00000,98.000000,1125.000000,0.000000,652.000000,821.000000,4.000000,0.000000,0.000000,2.000000
50%,291821.000000,9769.000000,85.000000,3.000000,10.000000,5.000000,4.000000,8.00000,1.000000,0.000000,10.000000,0.000000,225.00000,613.000000,1266.000000,0.000000,964.000000,1209.000000,6.000000,0.000000,0.000000,4.000000
75%,368163.000000,11450.000000,115.000000,4.000000,14.000000,7.000000,5.000000,10.00000,1.000000,0.000000,12.000000,0.000000,254.00000,718.000000,2473.000000,0.000000,1072.000000,1363.000000,9.000000,0.000000,0.000000,6.000000
max,883564.000000,52697.000000,542.000000,31.000000,127.000000,74.000000,27.000000,43.00000,5.000000,7.000000,45.000000,64.000000,400.00000,1037.000000,3828.000000,30.000000,1429.000000,2004.000000,63.000000,1.000000,1.000000,24.000000


In [0]:
dataBase["DiagCorr"], dataBase["DiagComb"], dataBase["DiagCont"] = "NaN", "NaN", "NaN"
for i in dataBase.index:
  dataBase.loc[i, "DiagCorr"] = dataBase.loc[i, "Diagnostico"][0] if dataBase.loc[i, "Diagnostico"][0] != " " else "NaN"
  dataBase.loc[i, "DiagComb"] = dataBase.loc[i, "Diagnostico"][1] if dataBase.loc[i, "Diagnostico"][1] != " " else "NaN"
  dataBase.loc[i, "DiagCont"] = dataBase.loc[i, "Diagnostico"][-1] if dataBase.loc[i, "Diagnostico"][-1] != " " else "NaN"

In [0]:
for i in dataBase["Código"]:
  prov = dataBase[dataBase["Código"] == i]
  if len(prov) > 1:
    print(prov)

In [0]:
col_string = ["Oxi","Nit","Sul","Ful","Agu","Gli","Dil"]
string = "<0,10"
for col in col_string:
  for i in dataBase.index:
    if(type(dataBase.loc[i, col]) == type(string)):
      dataBase.loc[i, col] = float(dataBase.loc[i, col][1:].replace(",", ".")) 

In [0]:
#for col in col_string:
#  for i in dataBase.index:
#    print(type(dataBase.loc[i, "Oxi"]))

In [0]:
dataBase.head(2)

,Código,Tipo,Marca,Km Atual,Km Troca,Diagnostico,Data Troca,Fe,Cr,Pb,Cu,Sn,Al,Ni,Ag,Si,B,Na,Mg,Ca,Ba,P,Zn,Mo,Ti,V,K,Oxi,Nit,Sul,Ful,Agu,Gli,Dil,DiagCorr,DiagComb,DiagCont
0,65307:,OM 366\tMOTOR,RIMULA SMV,88898.0,14185.0,GGG,1998-09-07 00:00:00,87.0,15.0,7.0,8.0,8.0,2.0,0.0,0.0,7.0,0.0,196.0,586.0,1305.0,0.0,952.0,1182.0,0.0,0.0,0.0,4.0,0.03,0.06,0.21,0.13,0.1,0.08,2,G,G,G
1,68053:,OM 366\tMOTOR,RIMULA SMV,100314.0,14805.0,GYG,27/07/98,99.0,21.0,10.0,8.0,10.0,2.0,0.0,0.0,7.0,0.0,193.0,558.0,1284.0,0.0,949.0,1177.0,0.0,0.0,0.0,5.0,0.05,0.09,0.25,0.5,0.1,0.08,2,G,Y,G


In [0]:
dataBaseLucy = dataBase
dataBaseLucy = dataBaseLucy.drop(columns = ["Código","Tipo","Marca","Diagnostico","Data Troca"])

In [0]:
dataBaseCorrosao = dataBaseLucy.drop(columns = ["DiagComb","DiagCont"])
dataBaseCombustao = dataBaseLucy.drop(columns = ["DiagCorr","DiagCont"])
dataBaseContaminacao = dataBaseLucy.drop(columns = ["DiagCorr","DiagComb"])

In [0]:
dataBaseCorrosao = dataBaseCorrosao[np.arrdataBaseCorrosao["DiagCorr"]]

In [0]:
dataBaseCorrosao.head(2)

,Km Atual,Km Troca,Fe,Cr,Pb,Cu,Sn,Al,Ni,Ag,Si,B,Na,Mg,Ca,Ba,P,Zn,Mo,Ti,V,K,Oxi,Nit,Sul,Ful,Agu,Gli,Dil,DiagCorr
0,88898.0,14185.0,87.0,15.0,7.0,8.0,8.0,2.0,0.0,0.0,7.0,0.0,196.0,586.0,1305.0,0.0,952.0,1182.0,0.0,0.0,0.0,4.0,0.03,0.06,0.21,0.13,0.1,0.08,2,G
1,100314.0,14805.0,99.0,21.0,10.0,8.0,10.0,2.0,0.0,0.0,7.0,0.0,193.0,558.0,1284.0,0.0,949.0,1177.0,0.0,0.0,0.0,5.0,0.05,0.09,0.25,0.5,0.1,0.08,2,G


In [0]:
dataBaseCombustao.head(2)

,Km Atual,Km Troca,Fe,Cr,Pb,Cu,Sn,Al,Ni,Ag,Si,B,Na,Mg,Ca,Ba,P,Zn,Mo,Ti,V,K,Oxi,Nit,Sul,Ful,Agu,Gli,Dil,DiagComb
0,88898.0,14185.0,87.0,15.0,7.0,8.0,8.0,2.0,0.0,0.0,7.0,0.0,196.0,586.0,1305.0,0.0,952.0,1182.0,0.0,0.0,0.0,4.0,0.03,0.06,0.21,0.13,0.1,0.08,2,G
1,100314.0,14805.0,99.0,21.0,10.0,8.0,10.0,2.0,0.0,0.0,7.0,0.0,193.0,558.0,1284.0,0.0,949.0,1177.0,0.0,0.0,0.0,5.0,0.05,0.09,0.25,0.5,0.1,0.08,2,Y


In [0]:
dataBaseContaminacao.head(2)

,Km Atual,Km Troca,Fe,Cr,Pb,Cu,Sn,Al,Ni,Ag,Si,B,Na,Mg,Ca,Ba,P,Zn,Mo,Ti,V,K,Oxi,Nit,Sul,Ful,Agu,Gli,Dil,DiagCont
0,88898.0,14185.0,87.0,15.0,7.0,8.0,8.0,2.0,0.0,0.0,7.0,0.0,196.0,586.0,1305.0,0.0,952.0,1182.0,0.0,0.0,0.0,4.0,0.03,0.06,0.21,0.13,0.1,0.08,2,G
1,100314.0,14805.0,99.0,21.0,10.0,8.0,10.0,2.0,0.0,0.0,7.0,0.0,193.0,558.0,1284.0,0.0,949.0,1177.0,0.0,0.0,0.0,5.0,0.05,0.09,0.25,0.5,0.1,0.08,2,G


In [0]:
dataBaseCorrosao.loc[dataBaseCorrosao["DiagCorr"] == "G", "DiagCorr"] = 0
dataBaseCorrosao.loc[dataBaseCorrosao["DiagCorr"] == "Y", "DiagCorr"] = 1
dataBaseCorrosao.loc[dataBaseCorrosao["DiagCorr"] == "R", "DiagCorr"] = 2
dataBaseCombustao.loc[dataBaseCombustao["DiagComb"] == "G", "DiagCorr"] = 0
dataBaseCombustao.loc[dataBaseCombustao["DiagComb"] == "Y", "DiagCorr"] = 1
dataBaseCombustao.loc[dataBaseCombustao["DiagComb"] == "R", "DiagCorr"] = 2
dataBaseContaminacao.loc[dataBaseContaminacao["DiagCont"] == "G", "DiagCorr"] = 0
dataBaseContaminacao.loc[dataBaseContaminacao["DiagCont"] == "Y", "DiagCorr"] = 1
dataBaseContaminacao.loc[dataBaseContaminacao["DiagCont"] == "R", "DiagCorr"] = 2

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [0]:
#Normalizando pelo Z-score: 
for i in dataBaseCorrosao.columns:
  dataBaseCorrosao[i] = (dataBaseCorrosao[i] - dataBaseCorrosao[i].mean()) / (dataBaseCorrosao[i].std(ddof = 1))
for i in dataBaseCombustao.columns:
  dataBaseCombustao[i] = (dataBaseCombustao[i] - dataBaseCombustao[i].mean()) / (dataBaseCombustao[i].std(ddof = 1))
for i in dataBaseContaminacao.columns:
  dataBaseContaminacao[i] = (dataBaseContaminacao[i] - dataBaseContaminacao[i].mean()) / (dataBaseContaminacao[i].std(ddof = 1))

TypeError: ignored

In [0]:
#checar o covariancia da matriz
covMatriz = dataBase.cov()

In [0]:
#checar a correlação da matriz
corrMatriz = dataBase.corr()

In [0]:
#Os autovalores e autovetores já estão ordenados, de maneira descrescente em relação aos autovalores
autoValores, autoVetores = np.linalg.eigh(covMatriz)

In [0]:
autoValores

array([1.23362733e-03, 1.34518189e-03, 8.92485608e-02, 4.11822162e-01,
       4.67375670e-01, 1.98868932e+00, 3.73740052e+00, 4.74429777e+00,
       5.45077742e+00, 6.35529804e+00, 9.61132049e+00, 1.96761906e+01,
       2.38088937e+01, 4.28214842e+01, 1.77725758e+02, 7.79519603e+02,
       1.51170252e+03, 2.20079520e+03, 5.15234700e+04, 6.61621532e+05,
       2.16769486e+07, 1.46936249e+10])